In this notebook, we will get an overview of how to generate word vectors using the various word embedding methods discussed in the lecture

### Objectives:
- Implement Count Vectors with sklearn
- Implement TF-IDF Vectors with sklearn and gensim
- Train and save word2vec model with gensim
- Load Google's pretrained word2vec model
- Load Stanford's pretrained GloVe model

In [9]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.datasets import fetch_20newsgroups
import numpy as np

In [2]:
# fetch 4 news groups articles from this dataset
categories = ['alt.atheism', 'soc.religion.christian','comp.graphics', 'sci.med'] 

twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)
corpus = twenty_train.data[0:50]   # 

In [3]:
print(f"{corpus[0]}")

# This email seems to be about computer graphics
# This is not good neough for us to fit into any model.
# We have to do some processing

From: sd345@city.ac.uk (Michael Collier)
Subject: Converting images to HP LaserJet III?
Nntp-Posting-Host: hampton
Organization: The City University
Lines: 14

Does anyone know of a good way (standard PC application/PD utility) to
convert tif/img/tga files into LaserJet III format.  We would also like to
do the same, converting to HPGL (HP plotter) files.

Please email any response.

Is this the correct group?

Thanks in advance.  Michael.
-- 
Michael Collier (Programmer)                 The Computer Unit,
Email: M.P.Collier@uk.ac.city                The City University,
Tel: 071 477-8000 x3769                      London,
Fax: 071 477-8565                            EC1V 0HB.



In [4]:
# We use a count-vectorizer to do so
vectorizer = CountVectorizer() #initialize
# lowercase parameter
# token_pattern parameter
# 
X = vectorizer.fit_transform(corpus) # fit

print(f"Dimensions of Document-term matrix: {X.toarray().shape}")

# we only have 50 documents and only 3000+ different vocabulary

Dimensions of Document-term matrix: (50, 3075)


From the result above, we can see that the second dimension gives us the size of our vocabulary.

But why restrict ourselves to single words? We can pass an additional argument to the CountVectorizer() object to add n-grams to our vocabulary.

What is an n-gram?  It's just a collection of n consecutive words. For example:
"New", "York", "City", "subway" are all unigrams
"New York", "York City", "City subway" are bigrams
"New York City", "York City subway" are trigrams
"New York City subway" is a 4-gram

We can specify to include n-grams with the ngram_range argument.  This takes a tuple which specifies the range of n-grams that we should include (inclusively).

In [5]:
# Include unigrams, bigrams, and trigrams

vectorizer = CountVectorizer(ngram_range=(1,3)) # (1,3) is a range that generates 1, 2, 3-grams for you
X = vectorizer.fit_transform(corpus)

print(f"Dimensions of Document-term matrix: {X.toarray().shape}")  
# Now we can see that our number of columns exploded because we added 1,2,3-grams!

Dimensions of Document-term matrix: (50, 23397)


A common preprocessing step in many NLP applications is stop-word removal.
Common words like "a", "the", "and" often add a lot of noise, and don't typiccally contribute much to the task we are trying to solve.

CountVectorizer also comes equipped with a way of dealing with common English stop words!

In [6]:
vectorizer = CountVectorizer(ngram_range=(1,3), stop_words='english') 
# we can use the countvectorizer to remove stopwords for us here.
X = vectorizer.fit_transform(corpus)

print(f"Dimensions of Document-term matrix: {X.toarray().shape}")
# By removing the stop-words, we see that we reduced the number of words by 10k

Dimensions of Document-term matrix: (50, 14300)


In [ ]:
# now let's look at tf-idf!

# TF-IDF Vectors

Here we will demonstrate two ways to generate TF-IDF vectors with both sklearn and gensim.  It's good to be aware of both methods because depending on your specific workflow, one method might be easier than the other!

In [7]:
# For sklearn, it's VERY similar to how we did CountVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,3), stop_words='english') # just replace 'Count' with 'Tfidf'
X_tfidf = vectorizer.fit_transform(corpus)

In [10]:
# How do these two representations compare?
# Let's look at the first 50 dimensions of the first document to gain some intuition

np.set_printoptions(precision=3) # This just makes things a little easier to read

print(f"CountVector: {X.toarray()[0,0:50]}\n\n")

print(f"TFIDF: {X_tfidf.toarray()[0,0:50]}")

CountVector: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 2 2 1 1 0 0 0 0 0 0 0]


TFIDF: [0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.127 0.127 0.063 0.063 0.    0.    0.    0.    0.
 0.    0.   ]


Scikit-learn isn't our only option for doing TF-IDF.  Gensim is another popular library for many NLP tasks

In [11]:
# We can also use gensim. Not just Tfidf.

In [12]:
import gensim

In [13]:
# Tokenize the documents (run this line to tokenize your documents (split into BOW))
tokenized_docs = [gensim.utils.simple_preprocess(d) for d in corpus]  


# Create a Gensim Dictionary.  This creates an id to word mapping for everything in our vocbulary
# It is NOT the same as the dictionary object in the Python standard library

# This is LITERALLY dictionary. Not a dictionary object.
mydict = gensim.corpora.Dictionary()


# Create a Gensim Corpus object.  This creates a list of tuples for each document.
# The first element of the tuple is the word id, the second is the number of counts
mycorpus = [mydict.doc2bow(doc, allow_update=True) for doc in tokenized_docs]

In [14]:
# This creates the doc-term matrix as a numpy array.
# Typically these matrices are HUGE so, it's usuall not a great idea to create the full dense doc-term matrix.
# We do it here to illustrate that you can get the same info as we obtained in scikit-learn!

doc_term_matrix = gensim.matutils.corpus2dense(mycorpus, num_terms=len(mydict))

# When we see a "2dense", it is basically converting it into a matrix

In [15]:
doc_term_matrix

array([[2., 0., 0., ..., 2., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 3., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

In [16]:
# Creating a tf-idf model is very simple!
tfidf = gensim.models.TfidfModel(mycorpus)
tfidf_matrix = gensim.matutils.corpus2dense(tfidf[mycorpus], num_terms=len(mydict))

In [17]:
tfidf_matrix

array([[0.146, 0.   , 0.   , ..., 0.141, 0.   , 0.   ],
       [0.093, 0.   , 0.   , ..., 0.   , 0.   , 0.   ],
       [0.037, 0.   , 0.056, ..., 0.   , 0.   , 0.   ],
       ...,
       [0.   , 0.   , 0.   , ..., 0.   , 0.   , 0.072],
       [0.   , 0.   , 0.   , ..., 0.   , 0.   , 0.072],
       [0.   , 0.   , 0.   , ..., 0.   , 0.   , 0.072]], dtype=float32)

In [ ]:
# Now we are going to look at how Word2Vec and GloVe works!

# Word2Vec and GloVe

Word2Vec is a very powerful and useful word embedding method.  The math can get a little sticky, but luckily Gensim comes equipped with ways for us to train our own Word2Vec model, or load in a pre-trained word2vec model.  Let's check it out!

In [18]:
# Imagine we have a corpus of 9 documents

documents = ["Human machine interface for lab abc computer applications",
             "A survey of user opinion of computer system response time",
             "The EPS user interface management system",
             "System and human system engineering testing of EPS",
              "Relation of user perceived response time to error measurement",
              "The generation of random binary unordered trees",
              "The intersection graph of paths in trees",
              "Graph minors IV Widths of trees and well quasi ordering",
              "Graph minors A survey"]

tokenized_docs = [gensim.utils.simple_preprocess(d) for d in documents]


In [19]:
# This is a function in gensim that trains your own embedding for you
model = gensim.models.Word2Vec(tokenized_docs, size=10, # size refers to the desired dimension of our word vectors
                               window=2, # window refers to the size of our context window
                               min_count=1, 
                               sg=1)     # sg=1 means that we are using the Skip-gram architecture
                                        # sg=0 means that we are using the CBOW architecture


In [20]:
print(model['human'])

[ 0.041 -0.028  0.037 -0.05   0.007 -0.034  0.023 -0.014  0.005 -0.021]


C:\Users\xianj\Anaconda3\envs\metis\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


Training our own model with word2vec is pretty cool, but it requires us to have a large corpus of data.

Fortunately, research groups at Stanford and Google have made their **pre-trained word embeddings publicly available for us to use!**

Google's word2vec: https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit

GloVe:  https://nlp.stanford.edu/projects/glove/

Just note that these model's will require **~4 GB of RAM to fit in memory**

In [21]:
# Path to where the word2vec file lives
google_vec_file = '/Users/adam/Downloads/GoogleNews-vectors-negative300.bin'

In [23]:
# Load it!  This might take a few minutes...
model = gensim.models.KeyedVectors.load_word2vec_format(google_vec_file, binary=True)

# it is just loading all the different weights (embedding) into python

In [ ]:
# We can access individual word vectors using a dictionary-like syntax
model['Metis']

# this is already trained. basically what we are doing is just a look-up table of weights

In [ ]:
# Some cool results!

model.most_similar('meeting' ,topn=8)  # give the top8 words that are closest to meeting

In [ ]:
model.most_similar('president' ,topn=5)

In [ ]:
# Here's an analogy task!

model.most_similar(positive=['woman', 'king'], negative=['man'])

In [ ]:
model.most_similar(positive=['Paris', 'England'], negative=['London'])

Using GloVe with Gensim requires a little extra leg work, but it's not too bad.
The problem is that the file format that is publicly available doesn't play nice with Gensim.
Luckily, Gensim provides a handy method of converting it!

In [ ]:
glove_file = glove_dir = '/Users/adam/Downloads/glove.6B/glove.6B.100d.txt'

w2v_output_file = '/Users/adam/Downloads/glove.6B/glove.6B.100d.txt.w2v'

# The following utility converts file formats
gensim.scripts.glove2word2vec.glove2word2vec(glove_file, w2v_output_file)

In [ ]:
# Now we can load it!
model = gensim.models.KeyedVectors.load_word2vec_format(w2v_output_file, binary=False)

In [ ]:
# How does it compare to the previous examples we did with word2vec?
model.most_similar('meeting' ,topn=8)

In [ ]:
model.most_similar('president' ,topn=5)

In [ ]:
model.most_similar(positive=['woman', 'king'], negative=['man'])

# Your Turn!

- Using either word2vec or GloVe, what interesting analogies or relationships?

- Given a short piece of text (like a tweet) what strategies can you think of to create a "tweet vector"?